In [7]:
import numpy as np
from IPython.display import Math, display
import re

def parse_linear_system(equations):
    """
    Parse a system of linear equations and return coefficient matrix and constants vector
    """
    # Remove spaces and split by '='
    processed_equations = []
    for eq in equations:
        # Handle cases where there might be spaces around =
        parts = eq.replace(' ', '').split('=')
        if len(parts) == 2:
            left, right = parts
            try:
                processed_equations.append((left, float(right)))
            except ValueError:
                print(f"Warning: Could not parse constant in equation: {eq}")
                processed_equations.append((left, 0.0))
        else:
            print(f"Warning: Could not parse equation: {eq}")

    # Extract variables (assuming x, y, z, w, u, v, t, s)
    variables = set()
    for left, _ in processed_equations:
        # Find all variable names (letters)
        for char in left:
            if char.isalpha():
                variables.add(char)

    # Sort variables alphabetically
    variables = sorted(list(variables))

    # Create coefficient matrix
    n_equations = len(processed_equations)
    n_variables = len(variables)
    A = np.zeros((n_equations, n_variables))
    b = np.zeros(n_equations)

    # Parse each equation
    for i, (left, right) in enumerate(processed_equations):
        b[i] = right

        # Add '+' before negative terms for easier parsing
        eq_str = ""
        if left and left[0] != '-':
            eq_str = "+"
        eq_str += left.replace('-', '+-')

        # Split by '+' to get terms
        terms = [term for term in eq_str.split('+') if term]

        for term in terms:
            if not term:
                continue

            # Handle coefficients
            coeff = 1.0
            var = None

            # Check if term is just a number (constant)
            if all(c.isdigit() or c in '.-' for c in term) and not any(c.isalpha() for c in term):
                try:
                    b[i] -= float(term)  # Move to other side
                except ValueError:
                    pass
                continue

            # Extract coefficient and variable
            # Find where the variable starts
            var_start = -1
            for j, char in enumerate(term):
                if char.isalpha():
                    var_start = j
                    var = char
                    break

            if var_start >= 0:
                # Extract coefficient part
                coeff_part = term[:var_start] if var_start > 0 else ""
                if coeff_part == "" or coeff_part == "+":
                    coeff = 1.0
                elif coeff_part == "-":
                    coeff = -1.0
                else:
                    try:
                        coeff = float(coeff_part)
                    except ValueError:
                        coeff = 1.0

                var_index = variables.index(var)
                A[i, var_index] += coeff
            else:
                # Pure constant term
                try:
                    b[i] -= float(term)
                except ValueError:
                    pass

    return A, b, variables

def system_to_augmented_matrix(equations):
    """
    Convert system of equations to augmented matrix and display using MathJax
    """
    # Parse equations
    A, b, variables = parse_linear_system(equations)

    # Create augmented matrix
    augmented = np.column_stack([A, b])

    # Display original system
    print("Original System of Equations:")
    for i, eq in enumerate(equations):
        display(Math(f"\\text{{Equation {i+1}:}} \\quad {eq}"))

    print(f"\nVariables: {variables}")

    # Display augmented matrix in MathJax
    print("\nAugmented Matrix:")

    # Build matrix string for MathJax with correct vertical line placement
    matrix_str = "\\left(\\begin{array}{"

    # Add column specifiers: one for each variable column
    matrix_str += "c" * len(variables)

    # Add separator and column for constants
    matrix_str += "|c"
    matrix_str += "}"

    # Add matrix rows
    for i in range(augmented.shape[0]):
        row = []
        for j in range(augmented.shape[1]):
            # Format numbers nicely
            val = augmented[i, j]
            if val.is_integer():
                row.append(str(int(val)))
            else:
                # Format to avoid trailing zeros
                formatted = f"{val:.6f}".rstrip('0').rstrip('.')
                if formatted == "-0":
                    formatted = "0"
                row.append(formatted if formatted else "0")

        # Join all elements in the row
        matrix_str += " & ".join(row) + " \\\\ "

    matrix_str += "\\end{array}\\right)"

    display(Math(matrix_str))

    return augmented, variables

# Example usage
print("Linear System to Augmented Matrix Converter")
print("=" * 50)

# Example 1: Simple 2x2 system
equations1 = [
    "2x + 3y = 7",
    "x - y = 1"
]

print("Example 1:")
augmented1, vars1 = system_to_augmented_matrix(equations1)

print("\n" + "="*50 + "\n")

# Example 2: 3x3 system
equations2 = [
    "x + 2y - z = 3",
    "2x - y + 3z = 1",
    "3x + y + 2z = 4"
]

print("Example 2:")
augmented2, vars2 = system_to_augmented_matrix(equations2)

print("\n" + "="*50 + "\n")

# Example 3: System with missing variables
equations3 = [
    "x + z = 5",
    "2y - z = 1",
    "x + y = 3"
]

print("Example 3:")
augmented3, vars3 = system_to_augmented_matrix(equations3)

# Function to handle custom input
def solve_custom_system():
    print("\nCustom System:")
    print("Enter your system of equations (one per line). Press Enter twice when done.")
    equations = []
    while True:
        eq = input("Enter equation (e.g., '2x + 3y = 7'): ").strip()
        if not eq:
            break
        equations.append(eq)

    if equations:
        augmented, variables = system_to_augmented_matrix(equations)
        return augmented, variables
    else:
        print("No equations entered.")
        return None, None

# Uncomment the following lines to input your own system:
# print("\n" + "="*50)
# solve_custom_system()

Linear System to Augmented Matrix Converter
Example 1:
Original System of Equations:


<IPython.core.display.Math object>

<IPython.core.display.Math object>


Variables: ['x', 'y']

Augmented Matrix:


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>


Variables: ['x', 'y', 'z']

Augmented Matrix:


<IPython.core.display.Math object>